**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exploración y Curación de Datos**

*Edición 2021*

----

# Exploración de datos

En esta notebook, vamos a cargar el conjunto de datos de la [competencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

In [1]:
# Ejecutado desde Google Colab...

In [2]:
import matplotlib.pyplot as plt
import numpy
import pandas
import seaborn

seaborn.set_context('talk')

In [3]:
import plotly

# Make sure it's 4.14.3
plotly.__version__

'4.14.3'

In [4]:
# To update plotly, uncomment and run the following line:
!pip install plotly --upgrade

Requirement already up-to-date: plotly in /usr/local/lib/python3.7/dist-packages (4.14.3)


In [5]:
melb_df = pandas.read_csv('https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')
melb_df[:3]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


#### Breve estudio del dataset

In [6]:
melb_df.shape

(13580, 21)

In [7]:
melb_df.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [8]:
melb_df.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


## Datos adicionales

Vamos a aumentar los datos presentes en el conjunto dado con un dataset similar: las publicaciones de la plataforma AirBnB en Melbourne en el año 2018. El objetivo es estimar con mayor precisión el valor del vecindario de cada propiedad.

Para ello, utilizaremos un [conjunto de datos](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) de *scrapings* del sitio realizado por [Tyler Xie](https://www.kaggle.com/tylerx), también disponible en una competencia de Kaggle.

In [9]:
interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
]

airbnb_df = pandas.read_csv('https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv', usecols=interesting_cols)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
airbnb_df[:3]

,description,neighborhood_overview,street,neighborhood,city,suburb,state,zipcode,latitude,longitude,price,weekly_price,monthly_price
0,"House: Clean, New, Modern, Quite, Safe. 10Km f...",Very safe! Family oriented. Older age group.,"Bulleen, VIC, Australia",Balwyn North,Manningham,Bulleen,VIC,3105,-37.772684,145.092133,60,NaN,NaN
1,A large air conditioned room with queen spring...,This hip area is a crossroads between two grea...,"Brunswick East, VIC, Australia",Brunswick,Moreland,Brunswick East,VIC,3057,-37.766505,144.980736,35,200.0,803.0
2,RIGHT IN THE HEART OF ST KILDA! It doesn't get...,A stay at our apartment means you can enjoy so...,"St Kilda, VIC, Australia",St Kilda,Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,159,1253.0,4452.0


#### Breve estudio del dataset

In [11]:
airbnb_df.shape

(22895, 13)

In [12]:
airbnb_df.columns

Index(['description', 'neighborhood_overview', 'street', 'neighborhood',
       'city', 'suburb', 'state', 'zipcode', 'latitude', 'longitude', 'price',
       'weekly_price', 'monthly_price'],
      dtype='object')

In [13]:
airbnb_df.describe()

,latitude,longitude,price,weekly_price,monthly_price
count,22895.000000,22895.000000,22895.000000,2524.000000,1891.000000
mean,-37.826827,145.008457,148.004368,906.976228,3234.346906
std,0.066747,0.127711,210.883253,821.223782,2808.092144
min,-38.224427,144.484326,0.000000,91.000000,400.000000
25%,-37.852665,144.957692,71.000000,450.000000,1700.000000
50%,-37.817855,144.978402,111.000000,750.000000,2800.000000
75%,-37.803176,145.013441,165.000000,1066.000000,3912.500000
max,-37.482595,145.839127,12624.000000,11536.000000,54258.000000


In [14]:
# La advertencia previa se debe a que la columna zipcode tiene diferentes tipos.
airbnb_df.dtypes

description               object
neighborhood_overview     object
street                    object
neighborhood              object
city                      object
suburb                    object
state                     object
zipcode                   object
latitude                 float64
longitude                float64
price                      int64
weekly_price             float64
monthly_price            float64
dtype: object

Para poder realizar esta combinación de datos, tenemos que encontrar una columna que contenga información en común.

Utilizaremos el **zipcode**.

* ¿Qué desventajas tiene utilizar el *zipcode* para agregar información sobre la vecindad de cada propiedad en venta? ¿Se corresponde con lo que entendemos intuitivamente como vecindad?

**Rta.** El código postal no es preciso, ya que solo codifica una zona definida arbitrariamente (y cuya definición a quedado obsoleta con el transcurso del tiempo). Una propiedad puede compartir código postal con otra. No se corresponde con nuestro entendimiento de vecindad.

* ¿Qué otros métodos podrían utilizarse?

**Rta.** Utilizar propiamente la dirección. En Argentina, estaría codificado por *barrio*, *calle*, y *altura*. Utilizar la posición geográfica, es decir, *latitud* y *longitud*.

In [15]:
airbnb_df.zipcode.value_counts()[:5]

3000.0    2491
3000       876
3006.0     860
3182.0     579
3182       556
Name: zipcode, dtype: int64

 Sin embargo, este zipcode tiene tipos diferentes, que tendremos que estandarizar antes de continuar con la combinación de los datos.

In [16]:
airbnb_df['zipcode'] = pandas.to_numeric(airbnb_df.zipcode, errors='coerce')

### Valores en común

Antes de unir los conjuntos de datos, tenemos que asegurarnos que tengamos suficientes registros en común como para agregar información relevante.

In [17]:
intersection = numpy.intersect1d(
    airbnb_df.zipcode.values,
    melb_df.Postcode.values,
    assume_unique=False)

print("Airbnb unique zipcodes", len(airbnb_df.zipcode.unique()))
print("Sales unique postcodes", len(melb_df.Postcode.unique()))
print("Common codes", len(intersection))

Airbnb unique zipcodes 248
Sales unique postcodes 198
Common codes 191


In [18]:
pSales = melb_df.Postcode.isin(intersection).sum() / len(melb_df)
pAirbnb = airbnb_df.zipcode.isin(intersection).sum() / len(airbnb_df)

print(f'Records in Sales df with corresponding code from Airbnb df: {pSales}')
print(f'Records in Airbnb df with corresponding code from Sales df: {pAirbnb}')

Records in Sales df with corresponding code from Airbnb df: 0.9985272459499264
Records in Airbnb df with corresponding code from Sales df: 0.9302904564315353


### Exploración visual

Para terminar de asegurarnos de que las áreas representadas por ambos datasets son consistentes, podemos representar las coordenadas en un mapa usando *Plotly*.

Aunque estas visualizaciones no son completas en cuanto a la información geográfica que muestran, aproximadamente podemos ver que se tratan de zonas similares.

In [19]:
import plotly.express as px

color_col = 'YearBuilt'
# Una pequeña muestra de nuestro dataset (con año de construcción no nulo)
data = melb_df[melb_df[color_col].notna()].sample(300)

fig = px.scatter_geo(data, lat=data.Lattitude, lon=data.Longtitude, color=data[color_col])
fig.update_geos(fitbounds="locations")
fig.show()

In [20]:
import plotly.express as px

color_col = 'city'
# Una pequeña muestra de nuestro dataset (con ciudad no nula)
data = airbnb_df[airbnb_df[color_col].notna()].sample(300)

fig = px.scatter_geo(data, lat=data.latitude, lon=data.longitude, color=data[color_col])
fig.update_geos(fitbounds="locations")
fig.show()

## Operación merge

Es hora de combinar ambos conjuntos de datos. Una opción frecuentemente tentadora es aplicar la operación merge sin analizar demasiado la situación. Sin embargo, ¡la operación puede ocasionar una **explosión exponencial** de nuestro conjunto de datos!

¿Por qué la siguiente operación genera un dataframe con 2 millones de filas?

**Rta.** Por resultados anteriores, sabemos que hay muy pocos valores únicos (lo que significa que hay demasiados valores repetidos). La situación produce la generación de duplicados imprevistos.

In [21]:
# ¡Incorrecto! Duplicados imprevistos
len(melb_df.merge(airbnb_df, how='left', left_on='Postcode', right_on='zipcode'))

2139684

Recordemos que nuestro objetivo es agregar información sobre el valor del vecindario de la propiedad. ¿Qué información del conjunto de *Airbnb* nos sirve para representar esto?

* Promedio de `price` por zipcode (**22895** valores en *dataframe*)
* Promedio de `weekly_price` por zipcode (**2524** valores en *dataframe*)
* Promedio de `monthly_price` por zipcode (**1891** valores en *dataframe*)

Queda a interpretación de ustedes decidir cuál de todos ellos utilizar.

**Rta.** La columna de `price` es la más completa, ya que no posee valores `NaN`.

In [22]:
relevant_cols = ['price', 'weekly_price', 'monthly_price']

Cuando queremos agregar un único dato luego de agrupar un DataFrame, la operación es bastante sencilla.

Sólo tenemos que tener en cuenta que el nombre de la columna será el mismo, aunque su valor ahora esté agregado.

In [23]:
"""
Option 1: inefficient form.
The aggregation is calculated over all columns, and after is calculated
we filter only the columns we want.
"""
airbnb_df.groupby('zipcode').mean()[relevant_cols].reset_index()[:5]

,zipcode,price,weekly_price,monthly_price
0,2010.0,40.000000,NaN,NaN
1,2134.0,50.000000,NaN,NaN
2,2582.0,104.000000,NaN,NaN
3,3000.0,150.504307,918.738956,3407.204651
4,3001.0,132.500000,NaN,NaN


In [24]:
"""
Option 2: efficient form
First we filter out the columns we want, and then we only aggregate
those. Be careful to include the grouping column as well.
"""
airbnb_df[relevant_cols + ['zipcode']].groupby('zipcode').mean().reset_index()[:5]

,zipcode,price,weekly_price,monthly_price
0,2010.0,40.000000,NaN,NaN
1,2134.0,50.000000,NaN,NaN
2,2582.0,104.000000,NaN,NaN
3,3000.0,150.504307,918.738956,3407.204651
4,3001.0,132.500000,NaN,NaN


Agrupar y agregar utilizando más de un tipo de operación es un poco más complicado. *Pandas* nos provee de muchas formas para escribir la misma operación. Ninguna de ellas es muy intuitiva, y usualmente requiere renombrar las columnas para darles nombres más significativos.

In [25]:
relevant_cols = ['price', 'weekly_price', 'monthly_price', 'zipcode']

In [26]:
"""
Option 1
Pass a dictionary where the keys are the original columns to aggregate and
the values are the operations (or list of operations).
"""
airbnb_price_by_zipcode = airbnb_df[relevant_cols].groupby('zipcode')\
    .agg({'price': ['mean', 'count'],
          'weekly_price': 'mean',
          'monthly_price': 'mean'})\
    .reset_index()
# Flatten the two level columns
airbnb_price_by_zipcode.columns = [
  ' '.join(col).strip()
  for col in airbnb_price_by_zipcode.columns.values]
# Rename columns
airbnb_price_by_zipcode = airbnb_price_by_zipcode.rename(
    columns={'price mean': 'airbnb_price_mean',
             'price count': 'airbnb_record_count',
             'weekly_price mean': 'airbnb_weekly_price_mean',
             'monthly_price mean': 'airbnb_monthly_price_mean'}
)

airbnb_price_by_zipcode[:3]

,zipcode,airbnb_price_mean,airbnb_record_count,airbnb_weekly_price_mean,airbnb_monthly_price_mean
0,2010.0,40.0,1,NaN,NaN
1,2134.0,50.0,1,NaN,NaN
2,2582.0,104.0,1,NaN,NaN


In [27]:
"""
Option 2
Pass as argument name the new name of the column, and as value a tuple where
the first value is the original column and the second value is the operation.
"""
airbnb_price_by_zipcode = airbnb_df[relevant_cols].groupby('zipcode')\
    .agg(airbnb_record_count=('price', 'count'),
         airbnb_price_mean=('price', 'mean'),
         airbnb_weekly_price_mean=('weekly_price', 'mean'),
         airbnb_monthly_price_mean=('monthly_price', 'mean'))\
    .reset_index()

airbnb_price_by_zipcode[:3]

,zipcode,airbnb_record_count,airbnb_price_mean,airbnb_weekly_price_mean,airbnb_monthly_price_mean
0,2010.0,1,40.0,NaN,NaN
1,2134.0,1,50.0,NaN,NaN
2,2582.0,1,104.0,NaN,NaN


In [28]:
merged_sales_df = melb_df.merge(
    airbnb_price_by_zipcode,
    how='left',
    left_on='Postcode', right_on='zipcode'
)

merged_sales_df.sample(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,zipcode,airbnb_record_count,airbnb_price_mean,airbnb_weekly_price_mean,airbnb_monthly_price_mean
1692,Carlton North,658 Lygon St,2,h,880000.0,S,Nelson,18/06/2016,3.2,3054.0,2.0,1.0,0.0,142.0,NaN,NaN,Yarra,-37.79110,144.9688,Northern Metropolitan,3106.0,3054.0,138.0,121.398551,642.500000,1690.888889
554,Balwyn,87 Yerrin St,3,h,1910000.0,S,Jellis,17/09/2016,9.7,3103.0,3.0,1.0,2.0,697.0,NaN,NaN,Boroondara,-37.80440,145.0853,Southern Metropolitan,5682.0,3103.0,57.0,105.263158,606.750000,2111.250000
1313,Brunswick,46 Cassels Rd,2,h,860000.0,S,Nelson,16/04/2016,5.2,3056.0,3.0,1.0,2.0,352.0,NaN,1900.0,Moreland,-37.75660,144.9599,Northern Metropolitan,11918.0,3056.0,446.0,94.672646,618.758065,2308.978723
10408,Mitcham,14 Victory St,2,h,883000.0,S,Jellis,27/05/2017,17.2,3132.0,2.0,1.0,1.0,697.0,NaN,1960.0,Whitehorse,-37.81808,145.2057,Eastern Metropolitan,6871.0,3132.0,38.0,68.605263,NaN,NaN
2687,Footscray,2/58 Everard St,2,t,560000.0,S,Sweeney,12/11/2016,6.4,3011.0,2.0,1.0,1.0,111.0,NaN,NaN,Maribyrnong,-37.79690,144.8852,Western Metropolitan,7570.0,3011.0,198.0,95.601010,441.916667,1624.611111


In [29]:
len(merged_sales_df)

13580